In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import xarray as xr
from bibliotheque import *

## PARAMS

In [3]:
participants = ['PERSA01_1','GHIVA02_2','MONLI03_3']
rooms = ['Baseline','Take-off','Savannah','Canopy','Bats','Underground','Grassland','Sea','End of the world','Baseline_End']
dtypes = ['EEGL','EEGR','ECG','EDA','PPG']

In [40]:
srate = 1000

In [39]:
save = False

In [23]:
ecg_interesting_metrics = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF', 'HRV_S', 'HRV_CSI', 'HRV_CVI']
ppg_interesting_metrics = ['mean amplitude', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF', 'HRV_S', 'HRV_CSI', 'HRV_CVI']
eda_interesting_metrics = ['nb_peaks', 'mean_height', 'EDA_Symp', 'EDA_SympN']

## TOOLS

In [26]:
def eeg_to_metrics(eeg_sig, srate=srate):
    metrics = nk.eeg_power(eeg_sig, sampling_rate=srate)
    return metrics
    
def ecg_to_metrics(ecg, interesting_metrics, srate=srate, get_fci_signal=False):
    ecg = -ecg
    peaks, info_ecg = nk.ecg_peaks(ecg, sampling_rate=srate,method='neurokit', correct_artifacts=True)
    R_peaks = info_ecg['ECG_R_Peaks'] # get R time points
    metrics = nk.hrv(R_peaks, sampling_rate=srate)
    
    if get_fci_signal:
        diff_R_peaks = np.diff(R_peaks) 
        x = vector_time
        xp = R_peaks[1::]/srate
        fp = diff_R_peaks
        interpolated_hrv = np.interp(x, xp, fp, left=None, right=None, period=None) / srate
        fci = 60 / interpolated_hrv
        
        return metrics[interesting_metrics], fci
    else:
        return metrics[interesting_metrics]
    
def gsr_homemade_metrics(info, dict_symp):   
    nb_peaks = info['SCR_Peaks'].size
    mean_height = np.mean(info['SCR_Height'])
    mean_amp = np.mean(info['SCR_Amplitude'])
    mean_risetime = np.mean(info['SCR_RiseTime'])
    mean_recov_time = np.mean(info['SCR_RecoveryTime'])
    
    data = [nb_peaks, mean_height, mean_amp, mean_risetime, mean_recov_time, dict_symp['EDA_Symp'], dict_symp['EDA_SympN']]
    df = pd.Series(data=data, index = ['nb_peaks', 'mean_height', 'mean_amp', 'mean_risetime', 'mean_recov_time','EDA_Symp','EDA_SympN'])
    return df

def eda_to_metrics(eda_sig, interesting_metrics, srate=srate):
    __ , info = nk.eda_process(eda_sig, sampling_rate=srate, method='neurokit')
    dict_symp = nk.eda_sympathetic(eda_sig, sampling_rate = srate)
    metrics = gsr_homemade_metrics(info, dict_symp)
    return metrics[interesting_metrics]
    
def ppg_amplitude(sig, srate, show=False):
    idx_maxs = signal.find_peaks(sig, distance = srate/2)[0]
    idx_mins = signal.find_peaks(-sig, distance = None)[0]
 
    amplitudes = []
    val_creux = []
    val_sommets = []
    indices_creux = []
    indices_sommets = []
    for idx_sommet in idx_maxs:
        cond = idx_mins < idx_sommet
        if not sum(cond) == 0:
            idx_creux = idx_mins[cond][-1]
            value_creux = sig[idx_creux]
            value_sommet = sig[idx_sommet]
            
            
            val_creux.append(value_creux)
            val_sommets.append(value_sommet)
            indices_sommets.append(idx_sommet)
            indices_creux.append(idx_creux)
        
            amplitude = value_sommet - value_creux
            amplitudes.append(amplitude)
    # print(np.mean(amplitudes))
    

    df = pd.DataFrame(np.array([np.arange(1,len(val_sommets)+1 , 1), indices_sommets , indices_creux, val_sommets, val_creux, amplitudes]).T, columns = ['cycle','idx_max','idx_min','val_max','val_min','amplitude'])

    if show:
        plt.figure()
        plt.plot(sig)
        plt.plot(indices_sommets, val_sommets, 'x')
        plt.plot(indices_creux, val_creux, 'o')
        plt.show()
        
    return np.mean(amplitudes)

def ppg_to_metrics(ppg_sig, interesting_metrics, srate=srate): 
    peaks = nk.ppg_findpeaks(ppg_sig, sampling_rate=srate, method='elgendi', show=False)
    metrics = nk.hrv(peaks, sampling_rate=srate, show=False)
    amp = ppg_amplitude(ppg_sig, srate)
    metrics.insert(0, 'mean amplitude', amp)
    return metrics[interesting_metrics]

## LOAD DATA

In [7]:
da = xr.load_dataarray('../data_preprocessed/da_preprocessed.nc')

In [8]:
da

<xarray.DataArray (participant: 3, room: 10, dtype: 5, time: 120000)>
array([[[[ 2.18176509e+01,  2.43219515e+01,  2.07852637e+01, ...,
           1.12561842e+02,  1.09525188e+02,  9.94711188e+01],
         [-7.96891142e+01, -7.05606058e+01, -6.49429978e+01, ...,
           6.65251368e+01,  5.87020922e+01,  5.17070793e+01],
         [ 3.18680846e+00,  3.10475900e+00,  3.01521027e+00, ...,
           1.01080365e+02,  1.03278750e+02,  1.05264646e+02],
         [ 1.02505898e+02,  1.02502234e+02,  1.02498566e+02, ...,
           1.59521777e+02,  1.59502987e+02,  1.59484215e+02],
         [ 2.39515227e+01,  2.37509254e+01,  2.35447851e+01, ...,
          -2.14855204e+01, -2.10762900e+01, -2.06572035e+01]],

        [[-1.95357543e+02, -2.27077629e+02, -2.36121611e+02, ...,
           1.32220066e+02,  9.29632725e+01,  4.96371498e+01],
         [ 1.43393311e+02,  1.35409881e+02,  1.32841369e+02, ...,
          -2.19619013e+02, -2.04404065e+02, -1.84674814e+02],
         [-5.67615823e+01, -6.17053658e+01, -6.66241456e+01, ...,
           6.13386623e+00,  5.88923714e+00,  5.64710657e+00],
         [ 1.05877953e+02,  1.05875758e+02,  1.05873544e+02, ...,
           9.39333924e+01,  9.39324694e+01,  9.39315334e+01],
         [-1.84333072e+01, -1.84152671e+01, -1.83972762e+01, ...,
...
           4.85919584e+02,  4.88220650e+02,  5.00744710e+02],
         [ 5.36564377e+02,  5.43879740e+02,  5.51224612e+02, ...,
          -5.55643080e+02, -5.56180175e+02, -5.51469724e+02],
         [-4.50626586e+00, -3.68445779e+00, -2.87767963e+00, ...,
          -3.17155030e+01, -3.13076913e+01, -3.09123675e+01],
         [ 1.29031764e+02,  1.29031380e+02,  1.29030994e+02, ...,
           2.62194895e+02,  2.62196725e+02,  2.62198561e+02],
         [ 9.94600500e+00,  1.03110925e+01,  1.06750787e+01, ...,
          -1.53833948e+00, -1.77139595e+00, -2.00025987e+00]],

        [[ 5.19271481e+02,  5.34736255e+02,  5.44470464e+02, ...,
           5.13455951e+02,  5.05854552e+02,  5.03720532e+02],
         [-5.49305153e+02, -5.52393473e+02, -5.54687783e+02, ...,
          -4.70248002e+02, -4.79973213e+02, -4.96476030e+02],
         [-3.05270316e+01, -3.01516834e+01, -2.97788229e+01, ...,
          -2.21006900e+00, -3.34285632e+00, -4.39814013e+00],
         [ 2.62200402e+02,  2.62202248e+02,  2.62204099e+02, ...,
           2.51976953e+02,  2.51976791e+02,  2.51976644e+02],
         [-2.22490119e+00, -2.44529174e+00, -2.66140514e+00, ...,
          -3.80250672e+00, -3.46961861e+00, -3.12959792e+00]]]])
Coordinates:
  * participant  (participant) object 'PERSA01_1' 'GHIVA02_2' 'MONLI03_3'
  * room         (room) object 'Baseline' 'Take-off' ... 'Baseline_End'
  * dtype        (dtype) object 'EEGL' 'EEGR' 'ECG' 'EDA' 'PPG'
  * time         (time) float64 0.0 0.001 0.002 0.003 ... 120.0 120.0 120.0

## DATA TO METRICS

In [32]:
eeg_concat = []
ecg_concat = []
eda_concat = []
ppg_concat = []

for participant in participants:
    print(participant)
    for room in rooms:
        print(room)
        for dtype in dtypes:
            # print(dtype)
            sig = da.loc[participant, room, dtype, :].values
            
            if dtype in ['EEGL','EEGR']:
                metrics = eeg_to_metrics(sig)
                metrics.insert(0, 'chan', dtype)
                metrics.insert(0, 'room', room)
                metrics.insert(0, 'participant', participant)
                eeg_concat.append(metrics)
            elif dtype == 'ECG':
                metrics = ecg_to_metrics(sig, interesting_metrics = ecg_interesting_metrics)
                metrics.insert(0, 'room', room)
                metrics.insert(0, 'participant', participant)
                ecg_concat.append(metrics)
            elif dtype == 'EDA':
                metrics = eda_to_metrics(sig, interesting_metrics = eda_interesting_metrics)
                metrics['room'] = room
                metrics['participant'] = participant
                eda_concat.append(metrics)
            elif dtype == 'PPG':
                metrics = ppg_to_metrics(sig, interesting_metrics = ppg_interesting_metrics)
                metrics.insert(0, 'room', room)
                metrics.insert(0, 'participant', participant)
                ppg_concat.append(metrics)     
            
eeg_metrics = pd.concat(eeg_concat).drop(columns = ['Channel'])
ecg_metrics = pd.concat(ecg_concat)
eda_metrics = pd.concat(eda_concat, axis = 1).T
ppg_metrics = pd.concat(ppg_concat)

PERSA01_1
Baseline
Take-off
Savannah
Canopy
Bats
Underground
Grassland
Sea
End of the world
Baseline_End
GHIVA02_2
Baseline
Take-off
Savannah
Canopy
Bats
Underground
Grassland
Sea
End of the world
Baseline_End
MONLI03_3
Baseline
Take-off
Savannah
Canopy
Bats
Underground
Grassland
Sea
End of the world
Baseline_End


In [38]:
if save:
    eeg_metrics.to_excel('../metrics/eeg_metrics.xlsx')
    ecg_metrics.to_excel('../metrics/ecg_metrics.xlsx')
    eda_metrics.to_excel('../metrics/eda_metrics.xlsx')
    ppg_metrics.to_excel('../metrics/ppg_metrics.xlsx')